<center><img src='https://drive.google.com/uc?export=view&id=1qJ8NqAZolTBQY7lN-deZ8xEsU3dlUiLz' width=200></center>


<h6><center></center></h6>

<h1>
<hr style=" border:none; height:3px;">
<center>Recherche d'Information et traitement de données massives </center>
    <center> Lab 8 : Apprentissage et Recherche d'information - Learning to Rank - Approche par points</center>
<hr style=" border:none; height:3px;">
</h1>




L'objectif de cette séance est la mise en oeuvre des approches pour la recherche d'information vue comme un problème d'apprentissage d'une fonction d'ordonnancement.

En particulier, dans ce premier TP vous mettrez en oeuvre l'**approche par points** modélisée comme un problème de regression. 

<center><img src='https://drive.google.com/uc?export=view&id=1IiT8Go8Mxhnv0Uj6aFgdyN-3TyVUKukC' width="500" height="500"></center>




### Partie 1 :  Prise en main des données

Nous travaillerons à partir du jeu de données mis en place par Chris Manning et Pandu Nayak pour leur très bon [cours](http://web.stanford.edu/class/cs276/) de recherche d'information et qui est accessible dans le répertoire [Data/pa3-data](./Data/pa3-data). 


<div class ="alert alert-block alert-success">
Comme d'habitude, pour ceux qui utilisent Colab, il sera nécéssaire de télécharger et dézipper l'archive associée au Lab.

1.   Dans le dossier Lab8_LearningToRank_1_2021 que vous avez téléchargé, vous trouverez une archive Archive_Lab8.zip.
2.   Ouvrir le panneau Fichiers de colab (c'est à dire cliquez sur le logo en forme de dossier à gauche) et cliquez sur le premier bouton en haut à gauche vous permettant de télécharger. Téléchargez ainsi l'Archive_Lab8.zip dans votre colab. Si besoin actualiser jusqu'à voir l'archive apparaître dans le panneau. 
3.    Exécutez la cellule de code suivante pour dézziper l'archive.
</div>

In [ ]:
!unzip Archive_Lab8

#### Description du jeu de données

Ce jeu de données contient **un ensemble d'apprentissage** constitué de paires `(requête,document)` et pour chaque paire d'un ensemble d'informations utiles pour ordonnancer les documents. Les données sont aussi séparées en un ensemble d'apprentissage et en un ensemble de test.

Plus précisemment, le répertoire [Data/pa3-data](./Data/pa3-data) contient :

+ Le fichier [pa3.signal.train](Data/pa3-data/pa3.signal.train) qui contient 731 requêtes et pour chaque requête une liste d'au plus 10 documents renvoyés par un moteur de recherche connu. La description détaillée de ce fichier est donnée plus loin.

+ Le fichier [pa3.rel.train](Data/pa3-data/pa3.rel.train) qui contient les jugements de pertinences donnés sous la forme d'un score de pertinence entre -1 et 3 pour chaque requête et chaque document associé à cette requête. Un extrait de ce fichier est donné ci-dessous.

````
======================================================================================
query: stanford aoerc pool hours
  url: http://events.stanford.edu/2014/February/18/ 0.0
  url: http://events.stanford.edu/2014/February/6/ 0.0
  url: http://events.stanford.edu/2014/March/13/ 0.0
  url: http://events.stanford.edu/2014/March/3/ 0.0
  url: http://med.stanford.edu/content/dam/sm/hip/documents/FreeFitnessWeek.pdf 0.0
  url: http://web.stanford.edu/group/masters/pool.html 1.0
  url: https://alumni.stanford.edu/get/page/perks/PoolAndGyms 1.5
  url: https://cardinalrec.stanford.edu/facilities/aoerc/ 2.0
  url: https://explorecourses.stanford.edu/search?view=catalog&filter-coursestatus-Active=on&page=0&catalog=&q=PE+128%3A+Swimming%3A+Beginning+I&collapse= 0.5
  url: https://glo.stanford.edu/events/stanford-rec-open-house 0.5
  
======================================================================================
````




L'extrait ci-dessus indique que pour la requête `stanford aoerc pool hours`, le document d'url `http://events.stanford.edu/2014/February/18/` a pour score de pertinence `0.0` et que celui d'url `https://alumni.stanford.edu/get/page/perks/PoolAndGyms` a pour score de pertinence `1.5`.


Les fichiers [pa3.signal.train](Data/pa3-data/pa3.signal.train) et [pa3.rel.train](Data/pa3-data/pa3.rel.train) sont utilisés comme **ensemble d'apprentissage**.

Le répertoire contient aussi deux fichiers qui seront utilisés comme **ensemble de test** :

+ Le fichier [pa3.signal.dev](Data/pa3-data/pa3.signal.dev) qui contient 124 requêtes et pour chaque requête une liste d'au plus 10 documents renvoyés par un moteur de recherche connu.
+ Le fichier [pa3.rel.dev](Data/pa3-data/pa3.signal.dev) qui contient les scores de pertinences pour les requêtes et les documents du fichier [pa3.signal.dev](Data/pa3-data/pa3.signal.dev).



#### 1.1 Description du fichier [pa3.signal.train](Data/pa3-data/pa3.signal.train)


Le fichier [pa3.signal.train](Data/pa3-data/pa3.signal.train) contient pour un ensemble de requêtes, les documents retournés pour cette requête et un ensemble d'information pour chacun de ces documents que nous allons utiliser pour construire notre ensemble de fichiers caractéristiques.


Le code ci-dessous permet d'afficher une partie du fichier [pa3.signal.train](Data/pa3-data/pa3.signal.train) et de visualiser l'information pour une requête donnée. **Executez le !**


In [ ]:
import os

data_dir = './Data/pa3-data'
filename = os.path.join(data_dir, "pa3.signal.train")
with open(filename, 'r', encoding = 'utf8') as f:
    print(f.read()[0:1000])
print("...")

On voit que pour chaque document associé à une requête sont donnés :
 + son **url** (`url`)
 + son **titre** (`title`)
 + un ou plusieurs **headers** (`header`).
 + un ensemble de **termes d'importance** (`body_hits`) dans le document avec pour chacun de ces termes une liste des positions de chacun de ces termes dans le document. Le nombre de positions permet donc d'avoir la frequence du terme dans le document.
 + la **longueur du corps du document** (`body_length`) qui représente le nombre de termes présents dans le corps du document.
 + un **score de type pagerank** (`pagerank`) qui est un entier entre 0 et 9 qui mesure la qualité du document indépendamment de la requête.
 
D'autres informations comme :
+ les **textes d'ancrage** (`anchor_text`) associés à la page et le **nombre d'ancrage** (`stanford_anchor_count`) associé à chaque texte d'ancrage et qui correspond au nombre d'ancrages sur le domaine stanford.edu qui porte ce texte d'ancrage.

Par exemple, si le texte d'ancrage est *stanford math department* et que le nombre d'ancrage associé est 9, cela signifie qu'il y a 9 liens vers le document (la page courante) pour lesquels le texte d'ancrage est *stanford math department*.

Si vous ne vous rappelez pas ce qu'est un texte d'ancrage, vous pouvez vous reporter au [cours sur le web](https://centralesupelec.edunao.com/mod/folder/view.php?id=49142).


<div class="alert alert-block alert-info"> 

Prenez le temps de regarder les données, puis répondez aux questions associées dans le sondage Wooclap
</div> 
 
[**Wooclap**](https://app.wooclap.com/JXDNGT/questionnaires/60a2166fd9d74311359c66d9)


#### 1.2 Accès et récupération des données.

Pour faciliter la réalisation de ce TP, plusieurs modules vous sont fournis dans le répertoire `base_classes` de `Utils`. Il y a notamment la fonction `load_train_data` et le module `Query` qui permettent de charger les requêtes du fichier [pa3.signal.train](Data/pa3-data/pa3.signal.train) sous la forme d'un dictionnaire de requêtes comme montré dans le code ci-dessous.

**Executez le code ci-dessous !**

In [ ]:
# Récupération des 731 requêtes et des documents associés du fichier pa3.signal.train

import os
import sys
from Utils.base_classes.load_train_data import load_train_data
from Utils.base_classes.query import Query

# Repertoire des données 
data_dir = './Data/pa3-data'
# Nom du fichier
file_name = os.path.join(data_dir, "pa3.signal.train")
# chargement du fichier
query_dict = load_train_data(file_name)
print("Récupération de {0} requêtes".format(len(query_dict)))

Vous devriez avoir récupérer les informations relatives aux requêtes du fichier `pa3.signal.train` dans le dictionnaire `query_dict`.

In [ ]:
assert (len(query_dict.keys()))==731

<div class="alert alert-block alert-danger">
<b>Conseil : </b>  si le test précédent renvoie une AssertionError, prevenez votre chargé de TD. 
</div>

#### 1.3 Comprendre de dictionnaire `query_dict`

Afficher le dictionnaire `query_dict` et essayez de comprendre sa structuration.

In [ ]:
query_dict

`query_dict` est un dictionnaire python dont :
+ chaque clé est un object de type `Query`(programmation orientée objet). 
+ la valeur associée à chaque clé est aussi un dictionnaire dont :
    + les clés sont les `url` de chaque document renvoyé pour la requête considérée.
    + la valeur associé à chaque clé est un objet de type `Document` (programmation orientée objet) avec des attributs correspondants aux différentes informations utiles pour décrire le document.
    
<div class="alert alert-block alert-danger">
<b>Important : </b>  La connaissance de la programmation orientée objet n'est pas nécéssaire pour ce LAB mais il est par contre indispensable de prendre le temps de bien comprendre les exemples ci-dessous pour bien comprendre comment manipuler les différentes informations.
</div>    
    

**Récupération des documents associés à une requête donnée**

La code ci-dessous montre un exemple de la récupération de l'information associée à une requête donnée, ici la requête `"stanford aoerc pool hours"`. On observe bien que cette information est renvoyée sous la forme d'un dictionnaire dont la clé est l'url du document et la valeur un dictionnaire avec l'ensemble des informations associées.

In [ ]:
# Récupération de l'information associée à une requête sous forme d'un dictionnaire de documents
query_dict[Query("stanford aoerc pool hours")]

**1.3.1 Ecrivez le code ci-dessous permettant de récupérer l'ensemble des documents associés à la requête `cardinal nights`.**

In [ ]:
## Recupération des documents associés à la requête "cardinal nights"
# TO COMPLETE



**Récupération du nombre de documents pour une requête donnée**


Il est alors facile de récupérer le nombre de documents associés à une requête dans cet ensemble d'apprentissage à l'aide de cette structure de dictionnaire. Il s'agit de la longueur de la liste de l'ensemble des clés du dictionnaire `query_dict`.
Le code ci-dessous vous montre comment récupérer le nombre de document pour la requête `"stanford aoerc pool hours"`.


In [ ]:
query_1 = query_dict[Query("stanford aoerc pool hours")]
print("Le nombre de documents pour la requête {0} est {1}".format("stanford aoerc pool hours",len(query_1)))

In [ ]:
# Affichage des urls des différents documents associés à une requête donnée

query_1 = query_dict[Query("stanford aoerc pool hours")]
for doc in query_1:
    print("Le document d'url {0} a été renvoyé pour la requête {1}".format(doc,"stanford aoerc pool hours"))

**1.3.2 Ecrivez le code permettant de récupérer le nombre de documents associés à la requête `cardinal nights`.**

In [ ]:
## Recupération du nombre de documents associés à la requête "cardinal nights"
# TO COMPLETE

**Récupération des différentes informations associées à un document**

Pour une requête donnée et un document associé, identifié par son url, il est ensuite possible de récupérer le document et les différentes informations sur ce dernier :
+ son titre
+ ses headers
+ ses termes d'importance et leurs positions dans le corps du document.
+ la longeur du corps du document
+ son page rank
+ d'éventuels textes d'ancrage.

Le code ci-dessous vous montre comment récupérer ces différentes informations pour la requête `"stanford aoerc pool hours"` et le document associé d'url `" http://events.stanford.edu/2014/February/18/"`.

In [ ]:
# Récupération d'un document
sample_doc = query_dict[Query("stanford aoerc pool hours")]['http://events.stanford.edu/2014/February/18/']
sample_doc

In [ ]:
# affichage du document
print("document:", sample_doc)

Pour récupérer les différentes informations du document, il suffit ensuite de suivre le syntaxe `var_document.nom_du_champ`. Par exemple, pour récupérer l'url du document précédent, stocké dans la variable `sample_doc`, il suffit d'utiliser la syntaxe `sample_doc.url` comme illustré ci-dessous.

In [ ]:
# récupération et affichage du contenu du champ url du document
url_doc = sample_doc.url
print("url :", url_doc)

Le code ci-dessous montre comment récupérer les headers du document référencé dans la variable `sample_doc`.

In [ ]:
# récupération et affichage du contenu du champ headers du document
headers_doc = sample_doc.headers
print("headers:", headers_doc)

Pour les termes d'importance, ils sont stockés sous la forme d'un dictionnaire python avec pour clé le terme lui-même et pour valeur associée à chaque clé, l'ensemble des positions du terme dans le corps du document. L'exemple ci-dessous montre comment récupérer les termes d'importance pour le document référencé dans la variable `sample_doc`.

In [ ]:
# récupération et affichage du contenu du champ body hits  du document
body_hits_doc = sample_doc.body_hits
print("body_hits:",body_hits_doc)

for term_hit in body_hits_doc:
    print("Le terme {0} apparait {1} fois dans le document de titre {2}".format(term_hit, len(body_hits_doc[term_hit]), sample_doc.title))

Comme vous le voyez dans l'exemple ci-dessus, il est possible de récupérer le nombre d'occurence du terme d'importance à partir de la liste des positions des documents.

**1.3.3 Ecrivez le code permettant de récupérer  pour le deuxième document associé à la requête `cardinal nights` ses informations utiles, à savoir son url, son titre, ses headers, ses termes d'importance et leurs fréquences respectives dans le corps du document**

In [ ]:
# TO COMPLETE

<div class="alert alert-block alert-info"> 

Une fois fait, complétez le Wooclap ci-dessous avec les différentes informations obtenues
</div> 
 
[**Wooclap**](https://app.wooclap.com/JXDNGT/questionnaires/60a242317dc46f112f7c5f54)

#### 1.4 Autres données utiles

D'autres données utiles sont fournies dans le répertoire [Data/pa3-data](./Data/pa3-data). Vous disposez notamment de :
+ un dictionnaire de termes [`terms.dict`](./Data/pa3-data/terms.dict) qui peut être chargé à l'aide du module `pickle` de python et qui associe à chaque terme un identifiant unique (`term_ID`) et inversement.
+ un dictionnaire de documents [`docs.dict`](./Data/pa3-data/docs.dict) qui peut être chargé à l'aide du module `pickle` de python et qui associe à chaque url de document un identifiant unique (`doc_ID`) et inversement.
+ un dictionnaire inversé [`BSBI.dict`](./Data/pa3-data/BSBI.dict) qui associe à chaque `term_ID` un triplet qui correspond à `(start_position_in_index_file, number_of_postings_in_list, length_in_bytes_of_postings_list)``.


**ATTENTION**, dans notre cas, la seule information utile est le nombre de postings dans la liste qui permet de récupérer la fréquence de documents `df` d'un terme. Cet index a été construit à l'aide de l'algorithme [BSBI](https://nlp.stanford.edu/IR-book/html/htmledition/blocked-sort-based-indexing-1.html) qui permet de gérer l'indexation de collections volumineuses.

Les codes ci-dessous montrent quelques exemples de manipulation de ces différents fichiers.


In [ ]:
# Manipulation des termes par leur IDs
import pickle as pkl
from Utils.base_classes.id_map import IdMap
import math

with open("./Data/pa3-data/terms.dict", 'rb') as f:
    terms = pkl.load(f)
    total_term_num = len(terms)
    print("Total Number of Terms is", total_term_num)

print('Le term ID de {0} est {1}'.format("radiology",terms["radiology"]))
print("Le terme d'ID {0} est {1}".format(1,terms[1]))


In [ ]:
# manipulation des documents par leur IDs

with open("./Data/pa3-data/docs.dict", 'rb') as f:
    docs = pkl.load(f)
    total_doc_num = len(docs)
    print("Total Number of Docs is", total_doc_num)
    
print("Le doc d'ID {0} a pour url {1}".format(1,docs[1]))
print("Le doc d'url {0} a pour docID {1}".format("0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html",docs["0/3dradiology.stanford.edu_patient_care_Case%2520studies_AVM.html"]))
print("Le doc d'ID {0} a pour url {1}".format(3452,docs[3452]))


Récupération des informations du dictionnaire inversé `BSBI.dict`

In [ ]:
# Manipulation de l'index inversé

with open('./Data/pa3-data/BSBI.dict', 'rb') as f:
    postings_dict, termsID = pkl.load(f)

print("Pour le terme {0} : {1}".format(terms[1],postings_dict[1]))
print("Le df (document frequency) du terme {0} est {1}".format(terms[1],postings_dict[1][1]))


####  Manipulation des dictionnaires.

En vous inspirant des exemples précédents, repondez aux questions ci-dessous.


#### 1.4.1 Dictionnaire de termes

Quel est l'ID du terme "administration" ?

In [ ]:
# TO complete

Quel terme a pour ID = 15000 ?

In [ ]:
# TO complete

#### 1.4.2 Dictionnaire de documents

Quel est l'ID du document d'url "https://uit.stanford.edu/service/webauth/twostep/push" ?

In [ ]:
# TO complete


Quel est l'url du document d'ID = 85000 ?

In [ ]:
# TO complete


Est-ce que la requête "alumni association benefits" est associé au document d'ID= 3800 dans le fichier d'apprentissage `pa3.signal.train`?

In [ ]:
# TO complete

<div class="alert alert-block alert-info"> 

Une fois fait, complétez le Wooclap ci-dessous avec les différentes informations obtenues
</div> 
 
[**Wooclap**](https://app.wooclap.com/JXDNGT/questionnaires/60a26e24a9c3c91151759da5)

## Partie 2 : construction d'un dictionnaire idf pour chaque terme

L'objectif de cette partie est de continuer à **prendre en main le jeu de données** en construisant un dictionnaire `{term: idf}` qui contient pour chaque terme son score `idf` et qui nous sera utile plus tard, pour l'approche d'apprentissage par paires.


Nous rappelons que pour un terme présent dans l'index inversé, on a :
$$idf(t) = \log_{10}\left(\frac{N}{df_t}\right)$$

Pour un terme non présent dans l'index inversé, on considère :

$$idf(t) = \log_{10}\left(\frac{N}{1.0}\right)$$

avec $N$ le nombre total de documents dans la collection.

**Compléter la fonction `build_IDF_dict()` ci-dessous. Cette dernière prend en paramètre le dictionnaire de termes, le dictionnaire de documents et l'index inversé et renvoie un dictionnaire qui associe à chaque terme sa valeur d'IDF.**

In [ ]:
def build_IDF_dict(terms_dict,BSBI_dict,doc_dict):
    IDF_dict={}
    # A completer
    return  IDF_dict


Testez votre fonction précédente en appliquant le code ci-dessous.


In [ ]:
IDF_voca=build_IDF_dict(terms,postings_dict,docs)
print(IDF_voca['radiology'])
# réponse attendue : environ 2.3631779024128257

**Quelle est la valeur d'IDF pour le terme "stanford" ?**

In [ ]:
# TO COMPLETE

## Partie 3 :  mise en place de l' approche par points



Comme nous l'avons vu en cours, la mise en oeuvre de l'approche par point consiste en :

   1. Une première étape dite de **feature engineering** et qui va consister à construire un vecteur représentatif pour chaque paire $x_{i,j} = (q_i,d_j)$ de l'ensemble d'apprentissage. En effet, chaque requête $q_i$ est associée à un ensemble de documents et pour chaque document $d_j$, il s'agira d'extraire un vecteur de caractéristiques caractérisant le couple $x_{i,j}= (q_i,d_j)$.
   2. La construction des données d'apprentissage supervisé en associant à chaque paire $x_{i,j}$ un label $y_{i,j}$ qui est le score de pertinence fourni dans le fichier [pa3.rel.train](Data/pa3-data/pa3.rel.train). 
   3. L'apprentissage d'une fonction d'ordonnancement sur l'ensemble d'apprentissage ainsi construit.
   4. L'évaluation du modèle appris sur les données de validation (ou de test).
***


#### Un problème de regression

Le problème d'ordonnancement consiste à apprendre une fonction $f$ telle que $f(x_{i,j})$ soit proche de $y_{i,j}$

Nous allons implémenter ici une instance très simple de l'approche par point qui consiste à représenter le problème comme un problème de **regression linéaire**. On va donc considérer une fonction $f$ qui donne un score à chaque paire $x =(q,d)$ selon $f(x) = wx+b$. Il s'agit donc d'apprendre le vecteur de poids ${w}$ et le terme de biais $b$ en minimisant la fonction de perte ci-dessous : 
\begin{equation}
\sum_{i=1}^m (f(x_{i})-y_{i})^2
\end{equation}
Il s'agit d'un problème de minimisation au sens des moindres carrés ordinaires.


### Partie 3 .1 : *feature engineering* - Construction du vecteur représentatif de $x=(q,d)$

Nous allons représenter chaque paire (requête,document) comme un vecteur de 4 dimensions où chaque dimension correspond à un des champs du document (titre, header, body et anchor) et comme valeur à chaque dimension le score `tf-idf` obtenu comme le produit scalaire de $q \cdot d_{f}$ avec $q$ le vecteur représentatif de la requête dans l'espace des termes et $d_f$ le vecteur représentatif du champ $f$ du document dans l'espace des termes.

Ainsi, si on considère $\vec{q}$ le vecteur représentatif de la requête et $\vec{d_f}$ le vecteur représentatif du champ $f$ construit sur l'espace des termes, il s'agit donc ici de construire $\vec{x}$, vecteur représentatif du couple comme :

$$\vec{x}=\left(\begin{array}{c}
 \vec{q} \cdot \vec{d_{\textrm{titre}}} \\
 \vec{q} \cdot \vec{d_{\textrm{header}}}  \\
 \vec{q} \cdot \vec{d_{\textrm{body}}} \\
 \vec{q} \cdot \vec{d_{\textrm{anchor}}} \\
\end{array} \right)$$

***

#### 3 .1. 1 - Représentation de la requête : construction du vecteur requête $ \vec{q}$.

Le vecteur requête sera construit en prenant en compte pour chaque terme :

 + sa fréquence logarithmique dans la requête.
 + son idf dans le corpus.

On ne fera pas de normalisation.
 
Ainsi, pour un terme $i$ présent dans la requête, son vecteur aura comme composante $i$ pour la requête $q$:

$$w_{iq}= (1 + \log_{10}(tf_{t_i,q})) \times \log (\frac{N}{df_{t_i}})$$
 
Ecrire le code permettant de calculer le vecteur d'une requête donnée. Il s'agit donc de compléter la fonction `def vect_query(q, IDF_voca,N)` ci-dessous qui prend en paramètre la requête `q` ainsi que le dictionnaire IDF et le nombre total de documents dans le corpus. Pour faciliter la suite des traitements, vous représenterez ce vecteur sous la forme d'un array `numpy`. Attention, certains termes de la requête peuvent être inconnus.

**Attention, on ne s'intéresse ici qu'aux termes de la requête.**



In [ ]:
def N=len(docs)

def vect_query(q,IDF_voca,N ):
    # A compléter
    return

Vous pouvez tester votre code avec la requête `'radiology radiology 3d'` et le code ci-dessous.

In [ ]:
# TO COMPLETE

Quelle représentation obtenez-vous pour la requête `"stanford robert crown library"`?

In [ ]:
# TO COMPLETE

Quelle représentation obtenez-vous pour la requête `""stanford aoerc pool hours""`?

In [ ]:
# TO COMPLETE

***

#### 3.1.2 - Représentation du document : construction du vecteur représentatif  de chaque champ de document $\vec{d_{\textrm{champ}}}$

Vous allez construire ici une représentation (en prenant uniquement en compte les termes de la requête) pour chaque champ des documents associés à une requête. Pour les champs multiples, on considèrera qu'il forme un seul gros document et on multipliera leur score `tf` par le compteur fourni si besoin (exemple ci-dessous `anchor_count`).

Par exemple, pour la requête $[\text{stanford aoerc pool hours}]^T$ et l'exemple ci-dessous
```python
  url: https://cardinalrec.stanford.edu/facilities/aoerc/
    ...
    anchor_text: gyms aoerc
      stanford_anchor_count: 3
    anchor_text: aoerc
      stanford_anchor_count: 13
    anchor_text: http cardinalrec stanford edu facilities aoerc
      stanford_anchor_count: 4
    anchor_text: arrillaga outdoor education and recreation center aoerc link is external
      stanford_anchor_count: 1
    anchor_text: the arrillaga outdoor education and research center aoerc
      stanford_anchor_count: 2
    anchor_text: aoerc will shutdown for maintenance
      stanford_anchor_count: 2
```

Le tf de l'<b>anchor</b> pour le document d'url `https://cardinalrec.stanford.edu/facilities/aoerc/` sera $[\text{4 25 0 0}]^T$ car il y a 4 *stanford_anchor_count* pour le terme “stanford” et 25 *stanford_anchor_count* pour le terme “aoerc”.

On calculera donc le vecteur associé à chaque champ avec, pour chaque terme présent dans la requête, sa fréquence logarithmique  dans le document soit pour le terme $t_i$ dans le document $d$ et la requête :

$$w_{id} = \frac{ 1 + \log_{10}(tf_{id})}{F_{\textrm{norm}}}$$
avec 
$F_{\textrm{norm}}$ un facteur de normalisation.

Vous diviserez tous les champs par le même facteur de normalisation, la longueur du corps du document.
Notez que certains documents ont une longueur de corps de 0, vous devrez donc les lisser d'une manière ou d'une autre. Une bonne stratégie consiste à ajouter une valeur, par exemple 500, à la longueur du corps de chaque document. Vous pouvez expérimenter avec cette valeur ou avec d'autres stratégies de lissage et les rapporter. Pour vous documenter sur les techniques de lissage, vous pouvez consulter le cours [Pour aller plus loin]()

**3.1.2.1 Completer la fonction ci-dessous qui pour une requête $q$ et un document $d$ associé calcule la représentation pour le champ `titre`.**
Pour faciliter la suite des traitements, vous représenterez ce vecteur de représentation sous la forme d'un array `numpy`.

In [ ]:
def vect_doc_title(q,d , query_dict):
    # To complete


Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"http://events.stanford.edu/2014/February/18/"`.

In [ ]:
# TO COMPLETE

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"https://cardinalrec.stanford.edu/facilities/aoerc/"`.

In [ ]:
# TO COMPLETE

**3.1.2.2 Completer la fonction ci-dessous qui pour une requête  𝑞  et un document  𝑑  associé calcule la représentation pour le champ `header`.**
Attention, il peut y avoir plusieurs headers et il faudra donc aggréger les différents headers si c'est le cas.
Pour faciliter la suite des traitements, vous représenterez ce vecteur de représentation sous la forme d'un array `numpy`.

In [ ]:
def vect_doc_header(q,d , query_dict):
    # To complete

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"http://events.stanford.edu/2014/February/18/"`.

In [ ]:
# TO COMPLETE

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"https://cardinalrec.stanford.edu/facilities/aoerc/"`.

In [ ]:
# TO COMPLETE

**3.1.2.3 Completer la fonction ci-dessous qui pour une requête  𝑞  et un document  𝑑  associé calcule la représentation pour le champ `body_hits`.**
Ici, vous vous intéressez donc qu'aux seuls termes communs entre les `body_hits`et la requête.
Pour faciliter la suite des traitements, vous représenterez ce vecteur de représentation sous la forme d'un array `numpy`.

In [ ]:
def vect_doc_body_hits(q,d , query_dict):
    # To complete

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"http://events.stanford.edu/2014/February/18/"`.

In [ ]:
# TO COMPLETE

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"https://cardinalrec.stanford.edu/facilities/aoerc/"`.

In [ ]:
# TO COMPLETE

**3.1.2.4 Completer la fonction ci-dessous qui pour une requête  𝑞  et un document  𝑑  associé calcule la représentation pour le champ `anchor`.**
Attention, il peut y avoir plusieurs anchors et il faudra donc aggréger les différents headers si c'est le cas.

Pour faciliter la suite des traitements, vous représenterez ce vecteur de représentation sous la forme d'un array `numpy`.

In [ ]:
def vect_doc_anchors(q,d , query_dict):
    # To complete

Tester votre fonction pour la requête "stanford aoerc pool hours" et le document d'url `"http://events.stanford.edu/2014/February/18/"`.

In [ ]:
# TO COMPLETE

Tester votre fonction pour la requête `"stanford aoerc pool hours"` et le document d'url `"https://cardinalrec.stanford.edu/facilities/aoerc/"`.

In [ ]:
# TO COMPLETE

***

### 3.1.3 Représentation du couple (document,requête) : vecteur représentatif  $x=(q,d)$
Ecrire une fonction qui pour un couple (document,requête) construit le vecteur de dimension 4 associé comme décrit précedemment. Il s'agit pour chacun des champs de calculer le produit scalaire entre la représentation de la requête et la représentation du champ du document.

Pour faciliter la suite des traitements, vous représenterez ce vecteur de représentation sous la forme d'un array `numpy`.

In [ ]:
def vect_query_doc(query, doc, query_dict):

# A compléter

Quelle est la représentation du couple $x=(\textrm{"stanford aoerc pool hours"}, \textrm{"https://cardinalrec.stanford.edu/facilities/aoerc/"}$ ?

In [ ]:
# To complete

***

### Partie 3. 2 : entraînement du modèle de régression linéaire 

Pour cette étape, nous allons nous servir de la bibliothèque [`scikit-learn`](https://scikit-learn.org/stable/) qui est une bibliothèque python dédiée à la fouille de données et à l’apprentissage automatique, développée par une équipe de l'INRIA (l'équipe Parietal de l'INRIA Paris Saclay). La communauté de développement de la bibliothèque est très active et c’est donc une bibliothèque qui évolue souvent et rapidement. Les principaux objets manipulés sont des objets de type array de numpy. C’est une bibliothèque qui possède de nombreuses fonctionnalités et elle est très utile dès lors que l’on souhaite mettre en place des algorithmes d'apprentissage.

Il vous faudra d'abord l'installer

In [ ]:
! pip3 install scikit-learn

Nous utiliserons en particulier le module [LinearRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) de cette bibliothèque dont un exemple d'utilisation est disponible [ici](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py) et qui met bien en avant le principe de scikit-learn selon lequel toute approche de regression ou de classification, ici représentée par la variable $clf$, utilise deux fonctions essentielles:
 + La fonction **clf.fit(data,target)** qui apprend un modèle à partir de données supervisée et le stocke dans la variable $\textrm{clf}$. `data` sont les données d'entrée et `target` les labels.
 + La fonction **clf.predict(newdata)** qui renvoie un tableau qui stocke, pour chaque nouvelle donnée en entrée, la classe prédite par le modèle appris.

Si vous n'avez jamais utilisé cette bibbliothèque, nous vous recommandons de prendre le temps de lire ce rapide tutorial [ici](https://scikit-learn.org/stable/tutorial/basic/tutorial.html).


***
#### 3.2.1 Calcul des caractéristiques sur le jeu de données pa3.

Nous allons ici mettre en oeuvre cette approche d'apprentissage sur le jeu de données pa3. On entrainera le modèle sur l'ensemble d'apprentissage (`pa3.signal.train` et `pa3.rel.train`) et on le testera sur l'ensemble de test (`pa3.signal.dev` et `pa3.rel.dev`).
La première étape consiste donc ici à appliquer l'étape précédente de feature engineering au jeu de données pa3. 

A l'aide des fonctions construites à l'étape précédente, calculer les réprésentations pour l'ensemble des données d'apprentissage. La fonction `get_relevance_score()` qui est donnée ici permet de récupérer sous la forme d'un dictionnaire les scores de pertinence des couples (requete,document).

In [ ]:
# Calculer les caractéristiques et les valeurs de pertinences pour les données d'apprentissage

train_signal_file = "./Data/pa3-data/pa3.signal.train"
train_rel_file = "./Data/pa3-data/pa3.rel.train"


def get_relevance_score(query_dict_rel):
    dict_rel = {}
    for query in query_dict_rel:
        dict_rel[query] = {}
        for result in query_dict_rel[query]:
            doc_url=result[:len(result)-4]
           # print(doc_url)
            relevance_score = result[len(result)-3:len(result)]
           # print(relevance_score)
            dict_rel[query][doc_url]=float(relevance_score)
    return  dict_rel 
    
dict_rel =get_relevance_score(query_dict_rel)




# A compléter


***

#### 3.2. 2 Entrainement du modèle d'ordonnancement.
Il s'agit ici d'apprendre le modèle de regression qui sera utilisé pour l'ordonnancement à l'aide des fonctionnalités de la bibliothèque `scikit-learn`.

In [ ]:
# Entrainer un modèle de regression linéaire
import numpy as np
from sklearn.linear_model import LinearRegression

# A completer

***
#### 3.2.3 Application du modèle de regression.

A l'aide de l'étape précédente, vous venez d'apprendre un modèle que vous pouvez maintenant utiliser sur de nouvelles données pour mettre en place un système de recherche d'information.
Pour cela, étant donnée une requête $q$ et l'ensemble des documents $(d_1,....d_m)$ qui contiennent des termes de la requête, il suffit d'appliquer le modèle $f$ appris à chaque couple $(q,d_i)$ qui va donc prédire un score de pertinence pour ce couple. Il suffira ensute d'ordonner par ordre de pertinence decroissant pour répondre à la requête.




Nous allons maintenant appliquer ce modèle à l'ensemble des données de test, c'est-à-dire les données du fichiers `pa3.signal.dev`.



In [ ]:
# Calculer les caractéristiques et les valeurs de pertinences pour les données de test

dev_signal_file = "./Data/pa3-data/pa3.signal.dev"
dev_rel_file = "./Data/pa3-data/pa3.rel.dev"

# a completer

In [ ]:
# Obtenir les prédictions sur l'ensemble de test

# A completer


***

### Partie 3. 3 : évaluation du modèle

Pour terminer ce travail, il est maintenant nécessaire d'évaluer le modèle de recherche ainsi appris. Vous allez ici évaluer votre modèle avec l'erreur quadratique moyenne et avec la mesure NDCG. En effet, comme nous avons des jugements de pertinence non-binaire, la métrique utilisée pour l'évaluation du système de recherche est le NDCG (Normalized Discounted Cumulative Gain). Vous pouvez prendre le temps de regarder le cours sur l'évaluation si vous ne vous rappeler plus de cette mesure.

Comme chaque requête a au plus 10 résultats, nous utilisons cette métrique sur les 10 premiers résultats de la recherche.

Pour une requête $q$ 

$$NDCG(q) = \frac{1}{Z} \sum_{m=1}^{p}\frac{2^{R(q,m)}-1}{log_{2}(1+m)}$$
Ici, $R(q, m)$ est le jugement de pertinence donné pour le document $m$ et la requête $q$. $Z$ est le facteur de normalisation et correspond à la valeur NDCG idéale que l'on obtient en triant les documents en ordre décroissant de pertinence et en calculant le NDCG  avec $Z=1$. $p$ est le nombre de documents qui sont retournés.

Pour un ensemble de requêtes $Q = \{q_1,...,q_m\}$ on prend la moyenne des NDCGs de chaque requête. 

On utilisera pour cela l'erreur quadratique scikit_learn : documentation [ici](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html#sklearn.metrics.mean_squared_error) et il vous faudra implémenter la mesure NDCG. Vous pouvez vous inspirer du module [NDCG](./Utils/base_classes/ndcg.py) écrit en orienté objet. 

In [ ]:
from sklearn.metrics import mean_squared_error
# A completer